# Function Collection to help solve linear systems problems

In [1]:
import sympy as sy
from sympy.integrals import inverse_laplace_transform as ilt
from sympy.integrals import laplace_transform as lt
from sympy import pretty_print as pp


In [2]:
# Matrix Functions

def round_expr(expr, num_digits):
    # https://www.thetopsites.net/article/54462201.shtml
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sy.Number)})

def matrix_info(matrix):
    ''' displays the determenant, inverse and charactersitic funtion of a matrix'''
    sMatrix = sy.Matrix(matrix)
    # char func is det(A- (Lambda x I)) = 0
    L = sy.symbols('L')
    LI = L * sy.eye(sMatrix.shape[0])
    charMatrix = sMatrix - LI
    print("A-Lamda x I: ")
    display(charMatrix)
    inverse = None
    if sy.det(sMatrix)!=0:
        inverse = sMatrix.inv()
    else:
        inverse = "Not invertable, det=0"
    display({
        "det": sMatrix.det(),
        "inverse":inverse,
        "charFunction": sy.latex(charMatrix.det())
    })
    
def get_eign(matrix):
    '''displays eignvalues and vectors of a matrix'''
    sMatrix = sy.Matrix(matrix)
    sym_eignvals = list(sMatrix.eigenvals().keys())
    sym_eignvects = [list(tup[2][0]) for tup in sMatrix.eigenvects()]
    display("Sympy eigenvalues: ", sym_eignvals)
    display("Sympy eigenvectors: ", sym_eignvects)


In [3]:
# State space model functions

    
def state_trans_matrix_inverse_laplace_method(a):
    '''warning can be slow, displays the state transisiton matrix assuming it exisits'''
    a = sy.Matrix(a)
    assert a.shape[0] == a.shape[1], "Must be a square matrix"
    s, t = sy.symbols('s t')
    invert = s*sy.eye(a.shape[0])-a
    display("We will now invert this", invert)
    invert = invert.inv()
    display("After we inverted", invert)
    out = ilt(invert, s, t)
#     out = sy.simplify(invert)
    display("Now we apply inverse Laplace transform to get the state transition matrix", out)

def state_trans_cayley_hamilton_method(a):
    '''Only implemented for a 2 by 2'''
    a = sy.Matrix(a)
    assert a.shape[0]==2 , "implemented for 2by2 only"
    eignvals = list(a.eigenvals().keys())
    t, a0, a1 = sy.symbols('t a0 a1')
    eq1 = sy.Eq(sy.exp(t*eignvals[0]), a0+a1*eignvals[0])
    eq2 = sy.Eq(sy.exp(t*eignvals[1]), a0+a1*eignvals[1])
    display(eq1,eq2)
    # Now we make the system of two variables into a solvable thing
    # https://stackoverflow.com/questions/31547657/how-can-i-solve-system-of-linear-equations-in-sympy

    e1 = sy.exp(t*eignvals[0]) -1*(a0+a1*eignvals[0])
    e2 = sy.exp(t*eignvals[1]) -1*(a0+a1*eignvals[1])
    sol = sy.solve([e1,e2], (a0,a1))
    display("a0 and a1 aka alpha 0 and alpha 1")
    display(sol[a0],sol[a1])
    ans = sol[a0] * sy.eye(2) + sol[a1]*a
    sy.simplify(ans)
    display(ans)
    

def state_trans_matrix(a,symbol = sy.symbols('t'), show_steps = True):
    a = sy.Matrix(a)
    state_trans = sy.exp(a*symbol)
    if show_steps:
        display("the state trans matrix is", state_trans)
    return state_trans

def state_var_homo_response(A_matrix, initial_condition_col,discrete = False, show_steps = True):
    '''x_homo at time t = state_trans_matrix * initial_cond '''
    A_matrix = sy.Matrix(A_matrix)
    initial_condition_col = sy.Matrix(initial_condition_col)
    if discrete:
        state_trans_mat = state_trans_matrix_discrete(A_matrix, show_steps = show_steps)
    else:
        state_trans_mat = state_trans_matrix(A_matrix, show_steps=show_steps)
        
    x_homo =  state_trans_mat * initial_condition_col
    if show_steps:
        display("Finding the homogenous response")
        display("A matrix",a,"Initial condition",initial_condition_col )
        display("x homogenous response at time t is ", x_homo)
    return x_homo

def state_var_forced_response(a, b, u, show_steps = True):
    '''Takes the A matrix and the B inputs column and returns the forced 
    response of the state space.
    '''
    a = sy.Matrix(a)
    b = sy.Matrix(b)
    # T is for tau
    t, T = sy.symbols('t T')
    state_trans_neg = state_trans_matrix(-1*a, T, show_steps = False)
    state_trans = state_trans_matrix(a, t, show_steps = False)
    # This is in case the u param is dependant on t, if it is we need to make it
    # into a T for the integration below
    try:
        u = u.replace(t,T)
    except:
        pass
    unevaluated_conv_integral = sy.Integral(state_trans_neg * b*u ,(T,0,t))
    conv_integral = unevaluated_conv_integral.doit() 
    forced_resp = state_trans * conv_integral
    if show_steps:
        display("Finding the forced response")
        display("A matrix",a, "B matrix",b)
        display("State transition matrix: ",state_trans)
        display("Negative state trans matrix", state_trans_neg)
        display("The convolution integral", unevaluated_conv_integral)
        display("Evaluated convolution integral", conv_integral)
        display("Foreced response is ", forced_resp)
        
    return forced_resp

def output_response(a,b,c,d,u, initial_condition_col, discrete = False, show_steps=True):
    '''a,b,c,d,u,ic matrices, and a bool if discrete'''
    a = sy.Matrix(a)
    b = sy.Matrix(b)
    c = sy.Matrix(c)
    d = sy.Matrix(d)
    initial_condition_col = sy.Matrix(initial_condition_col)
    # set show steps to false because it will be redundant
    homo_resp = state_var_homo_response(a,initial_condition_col,discrete = discrete, show_steps = False)
    forced_resp = []
    
    if discrete:
        forced_resp = state_var_forced_response_discrete(a, b,u, show_steps = False)
    else:
        forced_resp = state_var_forced_response(a, b,u, show_steps = False)

    full_resp_times_c_col = c * forced_resp + c * homo_resp
    
    y =  full_resp_times_c_col + d*u
    if show_steps:
        display("finding the output response")
        display("A matrix",a, "B matrix",b, "C matrix",c,"D matrix",d)
        display("initial condition", initial_condition_col)
        display("homogenous resp", homo_resp)
        display("forced resp", forced_resp)
        display("Full x response ie x homogenous + x forced", homo_resp + forced_resp)
        display("Full response times c col at time t (or k if discrete)", full_resp_times_c_col)
        display("the output y is ", y)
    
    return y

# def neat_output_resp_discrete():
#     sy.collect(round_expr(output_response(a,b,c,d,u,ic,discrete=True, show_steps=False),2)[0], k)
    

In [6]:
a = sy.Matrix([
    [1,1,0],
    [0,2,1],
    [0,0,1]
])
display(state_trans_matrix_inverse_laplace_method(a))

'We will now invert this'

Matrix([
[s - 1,    -1,     0],
[    0, s - 2,    -1],
[    0,     0, s - 1]])

'After we inverted'

Matrix([
[1/(s - 1), 1/(s**2 - 3*s + 2), 1/(s**3 - 4*s**2 + 5*s - 2)],
[        0,          1/(s - 2),          1/(s**2 - 3*s + 2)],
[        0,                  0,                   1/(s - 1)]])

'Now we apply inverse Laplace transform to get the state transition matrix'

Matrix([
[exp(t)*Heaviside(t), (exp(t) - 1)*exp(t)*Heaviside(t), (-t + exp(t) - 1)*exp(t)*Heaviside(t)],
[                  0,            exp(2*t)*Heaviside(t),      (exp(t) - 1)*exp(t)*Heaviside(t)],
[                  0,                                0,                   exp(t)*Heaviside(t)]])

None

In [7]:
def trans_matrix(a,b,c,d):
    '''returns the transfer matrix, which is the generlization of the transfer function'''
    a = sy.Matrix(a)
    b = sy.Matrix(b)
    c = sy.Matrix(c)
    d = sy.Matrix(d)
    assert a.shape[0] == a.shape[1], "A Must be a square matrix"
    s, t = sy.symbols('s t')
    invert = s*sy.eye(a.shape[0])-a
    display("We will now invert this", invert)
    invert = invert.inv()
    display("After we inverted", invert)
    G = c * invert * b 
    display("Answer of c * invert * b ", G)
    G = G +d
    display("Answer with +d so c * invert * b + d", G)
    G = sy.simplify(sy.together(G))
    display("simplified answer", G)
    return G



In [9]:
a = sy.Matrix([
    [0,1],
    [-2,-3]
])
state_trans_matrix(a)

'the state trans matrix is'

Matrix([
[   2*exp(-t) - exp(-2*t),    exp(-t) - exp(-2*t)],
[-2*exp(-t) + 2*exp(-2*t), -exp(-t) + 2*exp(-2*t)]])

Matrix([
[   2*exp(-t) - exp(-2*t),    exp(-t) - exp(-2*t)],
[-2*exp(-t) + 2*exp(-2*t), -exp(-t) + 2*exp(-2*t)]])

In [10]:
def z_transform(expr,n_symbol, start=0, stop=100):
    '''Uses the z transform defination to get its summation, use ".doit()" on the return value
    to evaluate the summation
    '''
    z,k = sy.symbols('z k')
    return sy.Sum(expr.replace(n_symbol, k)*z**(-k),(k,start,stop))
n,z = sy.symbols('n,z')
display(z_transform(3*sy.DiracDelta(n-2), n ,0, 100).doit())
a = sy.Matrix([
    [1,2,4],[2,5,0],[4,0,5]
])
# state_trans_matrix_inverse_laplace_method(a)
print("with the other functino")
display( sy.solve(z**2+6*z-15,z))
display( sy.solve(z**3-11*z**2+15*z+75,z))


3*DiracDelta(0)/z**2

with the other functino


[-3 + 2*sqrt(6), -2*sqrt(6) - 3]

[5, 3 - 2*sqrt(6), 3 + 2*sqrt(6)]

In [8]:
n,z = sy.symbols("n z")

# n = 1/4 * z**(-1)
# d = (1-0.5*z**(-1))*(1-0.25*z**(-1))
# n = z**2+5*z+6
# d = z+2

# n= z
# d = z-0.5
# sy.div(n,d)
# expr = z/((z+1)*(z+2))
# expr = n/d
# needs edge cases check, what happens when expr is just a number?
def inverse_Z_transform(expr, z_symbol = sy.symbols('z'), n_symbol= sy.symbols('n')):
    '''Uses the residue theorm explained here http://course.ece.cmu.edu/~ece491/lectures/L07/IZT_ContourIntegration.pdf'''
#     import ipdb; ipdb.set_trace()
    expr = sy.simplify(expr)
    f = sy.simplify(expr * z_symbol**(n_symbol-1))
    numer,denom = f.as_numer_denom()
    poles = sy.roots(denom)
    inverse = 0
    display(f)
    
    # looping through poles to find residues, I should use limits, but I am replacing instead
    assert len(poles) > 0, "There must be at least one pole, or answer is just a delta dirac"
    for p, multiplicty in poles.items():
        derivative_order = multiplicty - 1
        s = multiplicty
        res = (z_symbol-p)**s * f
        display(res)
        for i in range(derivative_order):
            res = sy.Derivative(res, z_symbol)
        display(res)
        factorial_div = sy.Rational(1,sy.factorial(s-1))
        res = sy.Mul(factorial_div, res)
        display("The residue equation looks like this", res)
        res = res.doit()
        res = res.replace(z_symbol,p)
        inverse += sy.simplify(res)
        display(f"The residue found for pole {p} is: ",sy.simplify(res))
    return inverse

def inverse_Z_transform_partial_fraction_method(expr, z_symbol = sy.symbols('z'), n_symbol= sy.symbols('n'), show_steps = True):
    expr_over_z = expr/z_symbol
    fracs = expr_over_z.apart()
    to_transform = sy.apart(sy.simplify(fracs * z_symbol))
    if show_steps:
        X = sy.Function("X")
        display("We will take the partial fraction of this",sy.Eq(X(z_symbol)/z_symbol, expr_over_z))
        display(sy.Eq(X(z_symbol)/z_symbol, fracs))
        display("Now we will take the Z-inverse of this, note that thepoles might have changes",sy.Eq(X(z_symbol), to_transform))
    return to_transform

def state_trans_matrix_Z_transform_method(a, show_steps = True):
    '''For discrete time'''
    a = sy.Matrix(a)
    assert a.shape[0] == a.shape[1], "Must be a square matrix"
    z, k = sy.symbols('z k')
    invert_me = z*sy.eye(a.shape[0])-a
    inverted = invert_me.inv()
#     out = ilt(inverted, z, k)
#      out = inverted.applyfunc(inverse_Z_transform)
    if show_steps:
        display("We will now invert this", invert_me)
        print(invert_me)
        display("After we inverted", inverted)
#         out = inverted.applyfunc(inverse_Z_transform_partial_fraction_method)

#         display("Now we apply inverse Laplace transform to get the state transition matrix", out)
 
    return out

# v = (z**2+z)/((z-1)**2)
# v = z/((z+1)*(z+2))
# v = (z**2+2*z)/(z**2-3*z+2) # got the wrong answer for this one
# inverse_Z_transform(v)
a= sy.Matrix([
[sy.Rational(-3,4) , sy.Rational(-1,4)],
    [sy.Rational(-1,4) , sy.Rational(-3,4)]
])
b = [
    [1],
    [0]
]
u=1
c = [[1,0]]
d = [4]
ic = [[0],
     [0]]



a = sy.Matrix([
    [0,1],
    [sy.Rational(-1,4), -1]
])

# state_trans_matrix_Z_transform_method(a)

In [60]:
# v = (z**2+z)/((z-1)**2)
# v = z/((z+1)*(z+2))
# v = (z**2+2*z)/(z**2-3*z+2) # got the wrong answer for this one
# # the one from the dgd
# # v = (z*(z+1))/((z+sy.Rational(1,2))*(z+sy.Rational(1,3))**2)
# display(v)
# # inverse_Z_transform_partial_fraction_method(v)
# inverse_Z_transform(v)


## sympy factor function gives wrong answer for problems like this
t = sy.Matrix([[z, -1], [sy.Rational(1,4), z + 1]])
# display(t/d)

display(t.inv())
d = t.det()
print("I want to factor this")
display(d)

print("After factoring")
display(sy.factor(d))

print(" \n The right answer I was expecting")
display((z+sy.Rational(1,2))**2)


Matrix([
[(4*z + 4)/(4*z**2 + 4*z + 1), -1/(4*(-z**2/4 - z/4 - 1/16))],
[     1/(4*(-z**2 - z - 1/4)),        4*z/(4*z**2 + 4*z + 1)]])

I want to factor this


z**2 + z + 1/4

After factoring


(2*z + 1)**2/4

 
 The right answer I was expecting


(z + 1/2)**2

In [5]:
#
# State space matrix discrete time functions
#
def state_trans_cayley_hamilton_discrete(a):
    '''Only implemented for a 2 by 2'''
    a = sy.Matrix(a)
    assert a.shape[0]==2 , "implemented for 2by2 only"
    eignvals = list(a.eigenvals().keys())
    k, a0, a1 = sy.symbols('k a0 a1')
    eq1 = sy.Eq(eignvals[0]**k, a0+a1*eignvals[0])
    eq2 = sy.Eq(eignvals[1]**k, a0+a1*eignvals[1])
    display(eq1,eq2)
    
    e1 = eignvals[0]**k -1*(a0+a1*eignvals[0])
    e2 = eignvals[1]**k -1*(a0+a1*eignvals[1])
    sol = sy.solve([e1,e2], (a0,a1))
    display("a0 and a1 aka alpha 0 and alpha 1")
    display(sy.simplify(sol[a0]),sy.simplify(sol[a1]))
    ans = sy.simplify(sol[a0]) * sy.eye(2) + sy.simplify(sol[a1])*a
    sy.simplify(ans)
    display(ans)
    
def state_trans_matrix_discrete(a, symbol = sy.symbols('k'), show_steps = True):
    a = sy.Matrix(a)
    state_trans =a ** symbol
    if show_steps:
        display("the state trans matrix is", state_trans)
    return state_trans

def state_var_forced_response_discrete(a, b,u, show_steps = True):
    '''Takes the A matrix and the B inputs column and returns the forced 
    response of the state space.
    '''
    a = sy.Matrix(a)
    b = sy.Matrix(b)
    # T is for tau
    k, i = sy.symbols('k i')
    state_trans_neg = state_trans_matrix_discrete(a, k-1-i, show_steps = False)
    state_trans = state_trans_matrix_discrete(a, k, show_steps = False)
    # This is in case the u param is dependant on k, if it is we need to make it
    # into an i for the summation below
    try:
        u = u.replace(k,i)
    except:
        pass
    unevaluated_conv_sum = sy.Sum(state_trans_neg * b*u ,(i,0,k-1))
    forced_resp = unevaluated_conv_sum.doit()
    if show_steps:
        display("Finding the forced response")
        display("A matrix",a, "B matrix",b)
        display("State transition matrix: ",state_trans)
        display("Negative state trans matrix", state_trans_neg)
        display("The convolution summation", unevaluated_conv_sum)
#         display("Evaluated convolution summation", conv_sum)
        display("Foreced response is ", forced_resp)
        
    return forced_resp


In [5]:
a,b,w = sy.symbols('a b w')
m = [
     [a,0],
    [0,b]
 ]
state_trans_matrix(m)
m2 = [[0,w], [-w,0]]
state_trans_matrix(m2)
print("Note that the above is the exponent form of trig functions sine and cosine")

'the state trans matrix is'

Matrix([
[exp(a*t),        0],
[       0, exp(b*t)]])

'the state trans matrix is'

Matrix([
[    exp(I*t*w)/2 + exp(-I*t*w)/2, -I*exp(I*t*w)/2 + I*exp(-I*t*w)/2],
[I*exp(I*t*w)/2 - I*exp(-I*t*w)/2,      exp(I*t*w)/2 + exp(-I*t*w)/2]])

Note that the above is the exponent form of trig functions sine and cosine


##### Example of equvilant state trans matrices using different methods

In [6]:
g,l,t = sy.symbols('g l t')
a = sy.Matrix([
    [0,-g/l],
    [1,0]
])
# a = sy.Matrix([
#     [0,1],[-2,-3]
# ])
# state_trans_matrix_inverse_laplace_method(a)
print("with cayley hamilton")
state_trans_cayley_hamilton_method(a)
display("We can also do this for an equivalent answer", sy.exp(a*t))
state_var_homo_response(a,[0,1])
print("Note that cosh(imaginary * x) =  cos (x), since the above has sqrt(-1) we can also write them in terms of sines and cosines and remove the imaginary term. We can see from below that their series equivlant is the same ")
cos_eq = sy.cos(sy.sqrt(g/l)*t)
cosh_eq = sy.cosh(sy.sqrt(-g/l)*t)
display("These two are equal, We can also see that their series below are equal", cos_eq, cosh_eq)
display(sy.series(cos_eq,t,n=3))
display(sy.series(cosh_eq,t,n=3))


with cayley hamilton


Eq(exp(-t*sqrt(-g/l)), a0 - a1*sqrt(-g/l))

Eq(exp(t*sqrt(-g/l)), a0 + a1*sqrt(-g/l))

'a0 and a1 aka alpha 0 and alpha 1'

cosh(t*sqrt(-g/l))

...

'These two are equal, We can also see that their series below are equal'

cos(t*sqrt(g/l))

cosh(t*sqrt(-g/l))

1 - g*t**2/(2*l) + O(t**3)

1 - g*t**2/(2*l) + O(t**3)

In [7]:
# example of a state space model solution
# x' = Ax + Bu
# y  = Cx + Du

a= sy.Matrix([
    [-2,0],
    [1,-1]
])
# ic = sy.Matrix([[2],[3]])
ic = [0,0]
b = sy.Matrix([
    [1],
    [0]
])
u = 5

c = [
    [2,1]
]

# state_var_homo_response(a, ic)

output_response(a,b,c,[0],u, ic)

'finding the output response'

'A matrix'

Matrix([
[-2,  0],
[ 1, -1]])

'B matrix'

Matrix([
[1],
[0]])

...

'Full response times c col at time t (or k if discrete)'

Matrix([[(exp(-t) - exp(-2*t))*(5*exp(2*t)/2 - 5/2) + 2*(5*exp(2*t)/2 - 5/2)*exp(-2*t) + (-5*exp(2*t)/2 + 5*exp(t) - 5/2)*exp(-t)]])

'the output y is '

Matrix([[(exp(-t) - exp(-2*t))*(5*exp(2*t)/2 - 5/2) + 2*(5*exp(2*t)/2 - 5/2)*exp(-2*t) + (-5*exp(2*t)/2 + 5*exp(t) - 5/2)*exp(-t)]])

Matrix([[(exp(-t) - exp(-2*t))*(5*exp(2*t)/2 - 5/2) + 2*(5*exp(2*t)/2 - 5/2)*exp(-2*t) + (-5*exp(2*t)/2 + 5*exp(t) - 5/2)*exp(-t)]])

In [8]:

# Obtaining the state transition matrix using different methods yields
# equivelant results
a = sy.Matrix([
[sy.Rational(-3,4) , sy.Rational(-1,4)],
    [sy.Rational(-1,4) , sy.Rational(-3,4)]
])
b = [
    [1],
    [0]
]
u=1
c = [[1,0]]
d = [4]
ic = [[0],
     [0]]
# matrix_info(a)
# state_trans = state_trans_matrix(a)
# state_trans_cayley_hamilton_method(a)
state_var_forced_response_discrete(a, b, u)
# state_var_homo_response(a, ic)
my_ans = sy.simplify(output_response(a,b,c,d,u,ic, discrete = True))
# display(state_trans_discrete_cayley_hamilton(a))
# state_trans_matrix_discrete(a)

######
# # comparing my solution against the solution manual, and mine is right!
# k = sy.symbols('k')
# sy.collect(sy.expand(sy.simplify(a**k)[0]),k).equals((-1)**k * (2**k +1)/2**(k+1))
# display(sy.expand(sy.simplify(a**k)[0]).equals((-1)**k * (2**k +1)/2**(k+1)))

# i = sy.symbols('i')
# book_ans = 4 + sy.Sum((-1)**(k-1-i)*(2**(k-1-i)+1)/(2**(k-i)), (i,0,k-1))
# display("answer in the book",book_ans)
# display("answer in the book",book_ans)
# book_ans = book_ans.doit()
# display("answer in the book",book_ans)

# display("my answer",my_ans[0])
# # display(book_ans.equals(my_ans[0]))
# print(my_ans[0].subs(k,1).equals(book_ans.subs(k,1)))
######

'Finding the forced response'

'A matrix'

Matrix([
[-3/4, -1/4],
[-1/4, -3/4]])

'B matrix'

Matrix([
[1],
[0]])

...

Matrix([
[-(-1)**k*(1/2 - (-1)**k/2)/2 - (-1/2)**k*(1/3 - (-2)**k/3)],
[-(-1)**k*(1/2 - (-1)**k/2)/2 + (-1/2)**k*(1/3 - (-2)**k/3)]])

'Full response times c col at time t (or k if discrete)'

Matrix([[-(-1)**k*(1/2 - (-1)**k/2)/2 - (-1/2)**k*(1/3 - (-2)**k/3)]])

'the output y is '

Matrix([[-(-1)**k*(1/2 - (-1)**k/2)/2 - (-1/2)**k*(1/3 - (-2)**k/3) + 4]])

In [9]:
### Q5
t, T = sy.symbols('t T')
a = [
    [0,0],
    [t,0]
]
b =  sy.Matrix([
    [1],
    [t]
])
u = t
c = [[1,t]]
d=[t]
ic = [[1],[1]]

matrix_info(a)
get_eign(a)
state_trans = state_trans_matrix(a)
state_var_forced_response(a, b,u)

# state_var_homo_response(a, ic)
my_ans = sy.simplify(output_response(a,b,c,d,u,ic))

### Checking my answer against the solution manual
# wtf the sol manual gave us two solutions to the 
# problem but they're both WRONG and not equal to each other

# book_sol =  1+t+3*t**2/2 + t**3/2 + t**4/3 + t**5/8
# book_sol2 = 3*t**2/2 + t**5/8 + t**4/3
# print(my_ans[0].equals(book_sol))
# print(my_ans[0].equals(book_sol2))
# book_sol.subs(t,1).equals(book_sol2.subs(t,1))

A-Lamda x I: 


Matrix([
[-L,  0],
[ t, -L]])

{'det': 0, 'inverse': 'Not invertable, det=0', 'charFunction': 'L^{2}'}

'Sympy eigenvalues: '

[0]

...

Matrix([
[                t**2/2 + 1],
[t**4/6 + t**3/2 + t**2 + 1]])

'Full response times c col at time t (or k if discrete)'

Matrix([[t**2/2 + t*(t**2 + 1) + t*(t**4/6 + t**3/2) + 1]])

'the output y is '

Matrix([[3*t**2/2 + t*(t**2 + 1) + t*(t**4/6 + t**3/2) + 1]])

## Example usage

In [10]:
# finding state transition matrix
a = sy.Matrix([
    [-1,0,0],
    [0,-4,4],
    [0,-1,0]
])
display("For ",a)
ans = state_trans_matrix(a)
x,y = sy.symbols('x y')
a = sy.Matrix([
    [x,y],
    [-y,x]
])
display("For ",a)

ans = state_trans_matrix(a)

'For '

Matrix([
[-1,  0, 0],
[ 0, -4, 4],
[ 0, -1, 0]])

'the state trans matrix is'

Matrix([
[exp(-t),                          0,                         0],
[      0, -2*t*exp(-2*t) + exp(-2*t),             4*t*exp(-2*t)],
[      0,               -t*exp(-2*t), 2*t*exp(-2*t) + exp(-2*t)]])

'For '

Matrix([
[ x, y],
[-y, x]])

'the state trans matrix is'

Matrix([
[     exp(t*x - I*t*y)/2 + exp(t*x + I*t*y)/2, I*exp(t*x - I*t*y)/2 - I*exp(t*x + I*t*y)/2],
[-I*exp(t*x - I*t*y)/2 + I*exp(t*x + I*t*y)/2,     exp(t*x - I*t*y)/2 + exp(t*x + I*t*y)/2]])

Finding the output for the following:


$$\dot{x} = \begin{bmatrix} -2 & 0 \\ 0 & -3 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
+
\begin{bmatrix} 1 \\ 1 \end{bmatrix}
u
\\
y = [-1,2]\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
$$

In [11]:
a = [
    [-2,0],
    [0,-3]
]
b = [[1],[1]]
c = [[-1,2]]
d = [0]
u=1
# initial condition
ic = [[0],[0]]
sy.expand(output_response(a,b,c,d,u,ic)[0])

'finding the output response'

'A matrix'

Matrix([
[-2,  0],
[ 0, -3]])

'B matrix'

Matrix([
[1],
[1]])

...

'Full response times c col at time t (or k if discrete)'

Matrix([[-(exp(2*t)/2 - 1/2)*exp(-2*t) + 2*(exp(3*t)/3 - 1/3)*exp(-3*t)]])

'the output y is '

Matrix([[-(exp(2*t)/2 - 1/2)*exp(-2*t) + 2*(exp(3*t)/3 - 1/3)*exp(-3*t)]])

1/6 + exp(-2*t)/2 - 2*exp(-3*t)/3

Discrete time example


$$ X(k+1)  =  A(k)X(k) + B(k)U(k)\\
Y(k) = C(k)X(k) + D(k)U(k)
$$

<br>

---

find the output for the following when k>=0 and u(k) = 1

$$ X(k) = \begin{bmatrix} 1 & 0 \\ k+1 & 0 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
+
\begin{bmatrix} 0 \\ 0 \end{bmatrix}
[u(k)]
\\
y = [1,\frac{1}{k+1}]\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
+[1][u(k)]
\\
X(0) = \begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$



In [12]:

k = sy.symbols('k')
a = [
    [1 , 0 ],
    [k+1, 0]
]
b = [
    [0],
    [0]
    ]
c = [
    [1, 1/(k+1)]
]
d = [1]
u = 1
ic = [
    [1],
    [1]
]
# output_response(a,b,c,d,u,ic, discrete=True)
state_var_forced_response_discrete(a,b,u)

'Finding the forced response'

'A matrix'

Matrix([
[    1, 0],
[k + 1, 0]])

'B matrix'

Matrix([
[0],
[0]])

...

Matrix([
[    1, 0],
[k + 1, 0]])**(-i + k - 1)

'The convolution summation'

Sum(Matrix([
[    1, 0],
[k + 1, 0]])**(-i + k - 1)*Matrix([
[0],
[0]]), (i, 0, k - 1))

'Foreced response is '

None

In [13]:
# example for solving discrete SS
# pdf link: https://nptel.ac.in/content/storage2/courses/108103008/PDF/module7/m7_lec4.pdf
a = sy.Matrix([
    [0, 1],
    [-0.21, -1]
])
b = [[0],[1]]
u = (-1)**k
d = [0]
c = [[0,1]]
ic = [[1],[0]]
# display(state_trans_discrete_cayley_hamilton(a))
# state_trans_discrete_matrix(a)
round_expr(state_var_forced_response_discrete(a,b,u),2)

# my_y = sy.expand(round_expr(output_response(a,b,c,d,u,ic,discrete=True, show_steps=True),2)[0], k)
# y_from_pdf = 0.475*(-3)**k - 5.3*(-0.7)**k + (-0.3)**k*(3.33)**k+5.825*(-0.7)**(k-1)*(1.43)**k
# display(y_from_pdf)

# display("If we compare the output from the pdf answer and my answer when we set k = 1 we get")
# display("pdf answer",y_from_pdf.subs(k,1))
# display("my answer",my_y.subs(k,1))
# display(" I think that my answer is right and their answer is wrong, because my function solved other questions correctly")

'Finding the forced response'

'A matrix'

Matrix([
[    0,  1],
[-0.21, -1]])

'B matrix'

Matrix([
[0],
[1]])

...

'The convolution summation'

Sum(Matrix([
[   (-1)**i*(2.5*(-0.3)**(-i + k - 1) - 2.5*(-0.7)**(-i + k - 1))],
[(-1)**i*(-0.75*(-0.3)**(-i + k - 1) + 1.75*(-0.7)**(-i + k - 1))]]), (i, 0, k - 1))

'Foreced response is '

Matrix([
[-3.57142857142857*(-0.3)**k*(-1.0 + 1.0*(-0.3)**(-k)*(-1)**k) + 8.33333333333333*(-0.7)**k*(-1.0 + 1.0*(-0.7)**(-k)*(-1)**k)],
[ 1.07142857142857*(-0.3)**k*(-1.0 + 1.0*(-0.3)**(-k)*(-1)**k) - 5.83333333333333*(-0.7)**k*(-1.0 + 1.0*(-0.7)**(-k)*(-1)**k)]])

Matrix([
[-3.57*(-0.3)**k*(-1.0 + 1.0*(-0.3)**(-k)*(-1)**k) + 8.33*(-0.7)**k*(-1.0 + 1.0*(-0.7)**(-k)*(-1)**k)],
[ 1.07*(-0.3)**k*(-1.0 + 1.0*(-0.3)**(-k)*(-1)**k) - 5.83*(-0.7)**k*(-1.0 + 1.0*(-0.7)**(-k)*(-1)**k)]])